# JEC profile plots

Poor man's coffea 0.7.1 mix of https://github.com/cms-jet/CoffeaJERC/blob/master/genL2L3.ipynb and [nanoevents.ipynb](https://github.com/CoffeaTeam/coffea/blob/master/binder/nanoevents.ipynb) to illustrate profile plots

In [ ]:
import awkward as ak
import numpy as np
import time
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

#fname = "root://cmsxrootd.fnal.gov//store/mc/RunIISummer19UL17NanoAOD/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8/NANOAODSIM/JMECustomTuples_106X_mc2017_realistic_v6-v1/280000/50221AB2-9CA3-C04A-A997-B01B901F079F.root"
#fname = "root://cmsxrootd.fnal.gov//store/mc/RunIISummer19UL17NanoAOD/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8/NANOAODSIM/JMECustomTuples_106X_mc2017_realistic_v6-v1/280000/0F7E67F1-5FCB-EC4B-A0B3-E0E9B98AFC43.root"
fname = "root://cmsxrootd.fnal.gov//store/mc/RunIISummer19UL17NanoAOD/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8/NANOAODSIM/JMECustomTuples_106X_mc2017_realistic_v6-v1/280000/0CEC4EFC-9CBD-B64C-8721-29D0CBB1F0AA.root"
events = NanoEventsFactory.from_root(fname, schemaclass=NanoAODSchema).events()

In [ ]:
from coffea import processor, hist
class FancyJECL2L3Processor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "Primary dataset")
        eta_axis = hist.Bin("eta", r"$\eta$", 20, -5, 5)
        pt_axis = hist.Bin("pt", r"$p_{T}$ [GeV]", 
                           np.array([5,10,15,20,25,30,35,40,45,50,60,70,80,90,
                                     100,120,140,160,180,
                                     200,250,300,350,400,450,500,
                                     600,700,800,900,1000,
                                     1500,2000,3000,4000,5000]))
        dr_axis = hist.Bin("dr", r"$\delta (\eta)$", 20, 0., 1)
        m_axis = hist.Bin("m", r"$p_{T}$ [GeV]", 200, 0, 500)
        r_axis = hist.Bin("r", "RECO / GEN response", 200, 0, 5)
        
        self._accumulator = processor.dict_accumulator({
            'pt':hist.Hist("Counts", dataset_axis, pt_axis),
            'eta':hist.Hist("Counts", dataset_axis, eta_axis),
            'dr':hist.Hist("Counts", dataset_axis, dr_axis),
            'r_pt_ptveta':hist.Hist("Counts", dataset_axis, pt_axis, eta_axis, r_axis),
            'r_m_ptveta':hist.Hist("Counts", dataset_axis, pt_axis, eta_axis, r_axis),
            'r_m_ptvm':hist.Hist("Counts", dataset_axis, pt_axis, m_axis, r_axis),
            'cutflow': processor.defaultdict_accumulator(int),
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        output['cutflow']['all events'] += len(events)
        #print(len(events))
        
        selectedEvents = events[
            (ak.num(events.Jet) > 2)
        ]

        #leading jet
        #print(selectedEvents.Jet[:,0])
        #leading two jets
        jet = selectedEvents.Jet[:,0:2]
        jet = ak.flatten(jet)
        #only with genmatch
        jet = jet[~ak.is_none(jet.matched_gen)]
        #only with good deltaR match
        jet = jet[jet.delta_r(jet.matched_gen)<0.2]
        
        ptresponse = jet.pt/jet.matched_gen.pt
        
        output['dr'].fill(dataset=selectedEvents.metadata["dataset"],
                            dr=jet.delta_r(jet.matched_gen))
        output['pt'].fill(dataset=selectedEvents.metadata["dataset"],
                            pt=jet.pt)
        output['eta'].fill(dataset=selectedEvents.metadata["dataset"], 
                                 eta=jet.eta)
        output['r_pt_ptveta'].fill( dataset=selectedEvents.metadata["dataset"], pt=jet.pt, eta=jet.eta, r=ptresponse)
        
        return output

    def postprocess(self, accumulator):
        return accumulator
   

In [ ]:
samples = {
    "QCDFlat": [fname]
}
tstart = time.time() 
output = processor.run_uproot_job(
    samples,
    "Events",
    FancyJECL2L3Processor(),
    processor.iterative_executor,
    {"schema": NanoAODSchema},
)
elapsed = time.time() - tstart
print(output)
print("Events/s:", output['cutflow']['all events']/elapsed)

In [ ]:
import mplhep as hep
import matplotlib.pyplot as plt
#import seaborn as sb

In [ ]:
EtaBins = output['r_pt_ptveta'].axis('eta')
EtaBinNums = len(output['eta'].values()[('QCDFlat',)])

PtBins = output['r_pt_ptveta'].axis('pt')
PtBinNums = len(output['pt'].values()[('QCDFlat',)])

In [ ]:
ax = hist.plotgrid(output['eta'], overlay="dataset", stack=False, density=True)
for i in range(EtaBinNums):
    print('Bin #' + str(i) + ': '+ str(EtaBins[i]))

In [ ]:
ax = hist.plotgrid(output['pt'], overlay="dataset", stack=False, density=True)
for i in range(PtBinNums):
    print('Bin #' + str(i) + ': '+ str(PtBins[i]))

In [ ]:
collect_etaranges = []
for bins in EtaBins:
    choicebin = PtBins[15]
    Hists = output['r_pt_ptveta'].sum('dataset').integrate('eta', bins).integrate('pt',choicebin)
    collect_etaranges.append(list(Hists.values().values())[0])
    title = r'$\eta$ range ' + str(bins) + r'; $p_T$ range ' + str(choicebin)
    ax = hist.plot1d(Hists)
    ax.set_title(title)
    plt.show()

In [ ]:
collect_ptranges = []
for bins in PtBins:
    choicebin = EtaBins[11]
    Hists = output['r_pt_ptveta'].sum('dataset').integrate('eta', choicebin).integrate('pt',bins)
    collect_ptranges.append(list(Hists.values().values())[0])
    title = r'$\eta$ range ' + str(choicebin) + r'; $p_T$ range ' + str(bins)
    ax = hist.plot1d(Hists)
    ax.set_title(title)
    plt.show()

In [ ]:
print(EtaBins)
for bins in EtaBins:
    title = r'$\eta$ range ' + str(bins)
    ax = hist.plot2d(output['r_pt_ptveta'].sum('dataset').integrate('eta', bins),xaxis='pt')
    ax.set_title(title)

In [ ]:
for bins in PtBins:
    title = r'$p_T$ range ' + str(bins)
    ax = hist.plot2d(output['r_pt_ptveta'].sum('dataset').integrate('pt', bins),xaxis='eta')
    ax.set_title(title)

In [ ]:
for bins in EtaBins:
    h = output['r_pt_ptveta'].sum('dataset').integrate('eta', bins)
    xaxis='pt'
    xaxis = h.axis(xaxis)
    yaxis = h.axes()[1]
    xoverflow='none'
    xedges = xaxis.edges(overflow=xoverflow)
    xcenters = xaxis.centers(overflow=xoverflow)
    vals = list(h.values().values())

    avs = [np.average(h.axes()[1].centers(), weights=b) if np.sum(b)>0 else 0. for b in vals[0]]
    #dummy
    avs_err = [0.]*len(avs)

    fig, ax = plt.subplots(1, 1)

    ax.set_xlabel(xaxis.label)
    ax.set_ylabel(yaxis.label)
    ax.set_xlim(xedges[0], xedges[-1])
    ax.set_ylim(0.5, 1.5)

    errbar = ax.errorbar(x=xcenters, y=avs, yerr=avs_err)
    
    title = title = r'$\eta$ range ' + str(bins)
    ax.set_title(title)
    plt.xscale("log")

In [ ]:
for bins in PtBins:
    h = output['r_pt_ptveta'].sum('dataset').integrate('pt', bins)
    xaxis='eta'
    xaxis = h.axis(xaxis)
    yaxis = h.axes()[1]
    xoverflow='none'
    xedges = xaxis.edges(overflow=xoverflow)
    xcenters = xaxis.centers(overflow=xoverflow)
    vals = list(h.values().values())

    avs = [np.average(h.axes()[1].centers(), weights=b) if np.sum(b)>0 else 0. for b in vals[0]]
    #dummy
    avs_err = [0.]*len(avs)

    fig, ax = plt.subplots(1, 1)

    ax.set_xlabel(xaxis.label)
    ax.set_ylabel(yaxis.label)
    ax.set_xlim(xedges[0], xedges[-1])
    ax.set_ylim(0.5, 1.5)

    errbar = ax.errorbar(x=xcenters, y=avs, yerr=avs_err)
    
    title = title = r'$p_T$ range ' + str(bins)
    ax.set_title(title)
    #plt.xscale("log")

Would be awesome to have a kind of projection function that gives "profile plots", e.g. 
* showing arithmetic mean +/- error, 
* median +/- errror (or interquartile range), 
* mode (e.g. from a Gaussian fit)
* violin plots etc.

In [ ]:
#print([output['pt'], output['eta']])
collections = [output['pt'].values()[('QCDFlat',)], output['eta'].values()[('QCDFlat',)]]
print(output['pt'].values()[('QCDFlat',)])

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.violinplot(collections, showmedians=True, showmeans=True, showextrema=True)
ax.set_xlabel('Outputs')
ax.set_ylabel('Counts')
ax.set_xticks([1,2])
ax.set_xticklabels([r'$p_T$', r'$\eta$'])
plt.show()

In [ ]:
'''Smaller range of collections (fewer violin plots at a time) if you want to see the details of the violin'''
a = (collect_ptranges[2], collect_ptranges[3])
b = (collect_etaranges[4], collect_etaranges[5])

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.violinplot(collect_ptranges, showmedians=True, showmeans=True, showextrema=True)
ax.set_xlabel(r'$p_T$ Bin Number')
ax.set_ylabel('Counts')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.violinplot(collect_etaranges, showmedians=True, showmeans=True, showextrema=True)
ax.set_xlabel(r'$\eta$ Bin Number')
ax.set_ylabel('Counts')
plt.show()